<a href="https://colab.research.google.com/github/Chamara-g/pre-datastorm-v1-0/blob/master/pre_data_storm_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd

In [34]:
#read train data set
df = pd.read_csv("drive/My Drive/FYP/dataset/credit_card_default_train.csv", index_col=0)
df.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
A4,100K,F,High School,Single,31-45,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
A5,200K,F,Graduate,Single,31-45,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
A6,1M,F,Graduate,Other,31-45,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [35]:
#train data set pre processing

#convet to string balance limit to float
df.Balance_Limit_V1 = (df.Balance_Limit_V1.replace(r'[KMB]+$', '', regex=True).astype(float) * df.Balance_Limit_V1.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
df.head()

#gender education marital age stringto int convert
df.Gender.replace(['M', 'F'], [1, 0], inplace=True)
df.EDUCATION_STATUS.replace(['Graduate', 'High School', 'Other'], [2, 1, 0], inplace=True)

#df.PAY_JULY.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_AUG.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_SEP.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_OCT.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_NOV.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_DEC.replace([-2, -1], [11, 13], inplace=True)

df.MARITAL_STATUS.replace(['Other', 'Single'], [1, 0], inplace=True)
df.AGE.replace(['Less than 30', '31-45', '46-65','More than 65'], [3, 2, 1, 0], inplace=True)

df.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [36]:
X = df.drop("NEXT_MONTH_DEFAULT",1)   #Feature Matrix
y = df["NEXT_MONTH_DEFAULT"]          #Target Variable
X

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A29996,1000000.0,0,1,0,2,0,0,0,0,0,0,436470,445403,481323,203289,72157,36914,19635,46200,11557,7039,11550,2310
A29997,400000.0,0,1,1,2,-1,-1,-1,-1,0,0,3888,4223,8090,20741,11989,0,4243,8145,20785,298,0,0
A29998,100000.0,0,0,1,2,4,3,2,-1,0,0,8235,7752,6371,48228,47544,44715,0,0,50820,9702,4620,7161


**Find Gradient of DUE_AMT**

In [37]:
DUE_X = X[["DUE_AMT_JULY", "DUE_AMT_AUG", "DUE_AMT_SEP", "DUE_AMT_OCT", "DUE_AMT_NOV", "DUE_AMT_DEC"]]
DUE_X.head()

,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC
Client_ID,,,,,,
A2,3248,3389,6004,39418,162772,-13982
A3,353351,151818,26948,43530,80811,124590
A4,16681,16082,15477,0,0,0
A5,90457,92848,95193,97309,100353,102740
A6,429556,419466,429785,435354,445271,453899


In [0]:
from statistics import mean

def best_fit_slope(xs,ys):
    m = (((mean(xs)*mean(ys)) - mean(xs*ys)) /
         ((mean(xs)**2) - mean(xs**2)))
    return m

In [39]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in DUE_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.DUE_AMT_JULY, rows.DUE_AMT_AUG, rows.DUE_AMT_SEP, rows.DUE_AMT_OCT, rows.DUE_AMT_NOV, rows.DUE_AMT_DEC] 

    xs = np.array([1,2,3,4,5,6], dtype=np.float64)
    ys = np.array(my_list, dtype=np.float64)

    # append the list to the final list 
    Row_list.append(best_fit_slope(xs,ys)) 
  
# Print the list 
#print(Row_list) 

X["DUE_AMT_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857


**Find Gradient of PAID_AMT**

In [40]:
PAID_X = X[["PAID_AMT_JULY", "PAID_AMT_AUG", "PAID_AMT_SEP", "PAID_AMT_OCT", "PAID_AMT_NOV", "PAID_AMT_DEC"]]
PAID_X.head()

,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,
A2,3437,6004,39418,162772,0,538165
A3,151818,46200,43530,80811,942,33666
A4,0,0,0,0,0,0
A5,3855,3890,3696,4620,4049,3918
A6,0,20790,16170,17325,16401,17325


In [41]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAID_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.PAID_AMT_JULY, rows.PAID_AMT_AUG, rows.PAID_AMT_SEP, rows.PAID_AMT_OCT, rows.PAID_AMT_NOV, rows.PAID_AMT_DEC] 

    xs = np.array([1,2,3,4,5,6], dtype=np.float64)
    ys = np.array(my_list, dtype=np.float64)

    # append the list to the final list 
    Row_list.append(best_fit_slope(xs,ys)) 
  
# Print the list 
#print(Row_list) 

X["PAID_AMT_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143,79399.485714
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714,-19692.942857
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143,-0.000000
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143,49.028571
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857,2131.800000


**Find Gradient of PAY**

In [42]:
PAY_X = X[["PAY_JULY", "PAY_AUG", "PAY_SEP", "PAY_OCT", "PAY_NOV", "PAY_DEC"]]
PAY_X.head()

,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC
Client_ID,,,,,,
A2,-1,-1,-1,-1,-1,-1
A3,0,-1,-1,-1,-1,0
A4,4,3,2,2,-2,-2
A5,2,0,0,0,0,0
A6,2,2,0,0,0,0


In [43]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAY_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.PAY_JULY, rows.PAY_AUG, rows.PAY_SEP, rows.PAY_OCT, rows.PAY_NOV, rows.PAY_DEC] 

    xs = np.array([1,2,3,4,5,6], dtype=np.float64)
    ys = np.array(my_list, dtype=np.float64)

    # append the list to the final list 
    Row_list.append(best_fit_slope(xs,ys)) 
  
# Print the list 
#print(Row_list) 

X["PAY_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143,79399.485714,-0.000000e+00
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714,-19692.942857,-1.522592e-16
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143,-0.000000,-1.285714e+00
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143,49.028571,-2.857143e-01
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857,2131.800000,-4.571429e-01


In [0]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [46]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(X.columns, clf.feature_importances_):
    print(feature)

('Balance_Limit_V1', 0.03387302440186589)
('Gender', 0.012675821289818088)
('EDUCATION_STATUS', 0.018495418211452567)
('MARITAL_STATUS', 0.011792629804836561)
('AGE', 0.0207425310182509)
('PAY_JULY', 0.0853506327833714)
('PAY_AUG', 0.039594632858586824)
('PAY_SEP', 0.026430670005612757)
('PAY_OCT', 0.02173109263987015)
('PAY_NOV', 0.019986003869435835)
('PAY_DEC', 0.017903693777208886)
('DUE_AMT_JULY', 0.05295060144135872)
('DUE_AMT_AUG', 0.048238418878939995)
('DUE_AMT_SEP', 0.04667049208509643)
('DUE_AMT_OCT', 0.04565907307025965)
('DUE_AMT_NOV', 0.045303098248273985)
('DUE_AMT_DEC', 0.04459221259735366)
('PAID_AMT_JULY', 0.046209531875967366)
('PAID_AMT_AUG', 0.043426049514192905)
('PAID_AMT_SEP', 0.04311978562981282)
('PAID_AMT_OCT', 0.03991772631243246)
('PAID_AMT_NOV', 0.03991978631569369)
('PAID_AMT_DEC', 0.04054046631781411)
('DUE_AMT_GRD', 0.06027696416498161)
('PAID_AMT_GRD', 0.055015531382052424)
('PAY_GRD', 0.0395841115054604)
